In [5]:
import pandas as pd
import re
import openai
import os
import time

In [ ]:
#we need to clean our input data
def clean_txt(text):
    text = str(text).strip() #remove leading and trailing whitespaces
    text = re.sub(r'\s+', ' ', text) #fix any unneccessary whitespace
    text = re.sub(r'^\x00-\x7F]+', '', text) #remove any characters non-ASCII
    return text
#used to split our description on each row into the new column titled items
def split(text):
    txt_lst = text.split("-")
    return txt_lst[0]

def split2(txt):
    txt_lst = txt.split("-")
    return txt_lst[1]
    
#format the data in a conversational format
def format_row(row):
    return f"Customer: Tell me about the {row['item']}.\nAgent: The {row['item']} is {row['description']}.\n"

#some of our descriptions lack correct sentence structure which is an issue for training
#client = openai.OpenAI(api_key=os.getenv("OPEMAI_API_KEY"))
def fix_grammar(txt):
    try:
        time.sleep(1)
        response = client.chat.completions.create(model="gpt-3.5-turbo", 
                                            messages=[{"role": "system", "content": "Improve grammar and sentence structure of the following text."},
                                                      {"role": "user", "content": txt}], 
                                            temperature=0.2,
                                            max_tokens=100)
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error: {e}")
        return txt


In [7]:
#read in our data
df = pd.read_csv("sample-data.csv")

#our item is hidden in our description so we need to extract that and create a new column
df["item"] = df["description"].apply(split)
df["description"] = df["description"].apply(split2)
df.dropna(subset=["description", "item"], inplace=True)
#clean up our data
for col in ["id", "description", "item"]:
    df[col] = df[col].apply(clean_txt)
    
df["description"] = df["description"].apply(fix_grammar)
#finally reformat it so it can be written into a text file
formatted = df.apply(format_row, axis=1) #apply to row via axis=1

In [8]:
with open("train.txt", "w", encoding = 'utf-8') as f:
    f.writelines(formatted)

In [11]:
from dotenv import load_dotenv
import os
#dotenv_path = os.path.abspath(os.path.join(os.path.dirname(__file__), '..', '..', '.env'))
dotenv_path = "Users/pc/Documents/CSPB/CSPB 3112/nanoGPT/.env"
load_dotenv(dotenv_path)
api_key = os.getenv("OPENAI_API_KEY")
print(api_key)

None
